# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine

import os
import nltk
# nltk.data.path.insert(0, os.path.expanduser("~/Downloads/nltk_data"))
# nltk.download('punkt', download_dir=nltk.data.path[0])
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chalee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('InsertTableName', engine)
X = df['message']
Y = df.drop(columns=['message', 'original', 'genre', 'child_alone'])
Y['related'] = Y['related'].map(lambda x: min(x, 1))

### 2. Write a tokenization function to process your text data

In [3]:
stem = SnowballStemmer("english").stem

def tokenize(text):
    words = word_tokenize(text.lower())
    words = [stem(w) for w in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize)),
    ("classifier", RandomForestClassifier())
#    ("classifier", MultiOutputClassifier(GradientBoostingClassifier()))
#    ("classifier", KNeighborsClassifier())
#    ("classifier", MultiOutputClassifier(AdaBoostClassifier()))
#    ("classifier", MLPClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
#X_train, Y_train = X, Y

pipeline.fit(X_train, Y_train)

C:\Users\Chalee\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pd.DataFrame(pipeline.predict(X_test),
                      index=Y_test.index,
                      columns=Y_test.columns)


for col in Y_pred.columns:
    print(">>> %s" % col)
    print(classification_report(Y_test[col], Y_pred[col]))

print(classification_report(Y_test, Y_pred))

>>> related
              precision    recall  f1-score   support

           0       0.58      0.45      0.51      1500
           1       0.85      0.90      0.87      5054

    accuracy                           0.80      6554
   macro avg       0.71      0.68      0.69      6554
weighted avg       0.78      0.80      0.79      6554

>>> request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5411
           1       0.84      0.34      0.48      1143

    accuracy                           0.87      6554
   macro avg       0.86      0.66      0.70      6554
weighted avg       0.87      0.87      0.85      6554

>>> offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0

C:\Users\Chalee\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



>>> infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6092
           1       0.25      0.00      0.00       462

    accuracy                           0.93      6554
   macro avg       0.59      0.50      0.48      6554
weighted avg       0.88      0.93      0.90      6554

>>> transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6241
           1       0.67      0.01      0.01       313

    accuracy                           0.95      6554
   macro avg       0.81      0.50      0.49      6554
weighted avg       0.94      0.95      0.93      6554

>>> buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6233
           1       0.70      0.02      0.04       321

    accuracy                           0.95      6554
   macro avg       0.83      0.51      0.51      6554
weighted avg      

C:\Users\Chalee\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Chalee\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    "vectorizer__ngram_range": [(1,1), (1,2), (1,5)],
    "vectorizer__stop_words" : [None, "english"],
    "vectorizer__use_idf": [True, False],
    "classifier__n_estimators": [10, 50, 100],
    "classifier__max_features": [0.01, 0.05, 0.1],
}
# Notes: max_features = 0.5 gives score ~0.63, "log2" gives 0.53.  But
# fitting times are too long with 0.5

cv = GridSearchCV(pipeline,
                  param_grid=parameters,
                  scoring='f1_micro',
                  n_jobs=-1)

cv.fit(X_train, Y_train)


C:\Users\Chalee\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = pd.DataFrame(cv.predict(X_test),
                      index=Y_test.index,
                      columns=Y_test.columns)


for col in Y_pred.columns:
    print(">>> %s" % col)
    print(classification_report(Y_test[col], Y_pred[col]))

print(classification_report(Y_test, Y_pred))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD

In [ ]:
def is_null(series):

    def is_null_base(s):
        res = 0 if s is None else 1
        return res
    
    final = pd.DataFrame(series.applymap(is_null_base))
    return final

In [ ]:
categorical_features = ['genre']
categorical_transform = OneHotEncoder()

original = ['original']
original_transform = FunctionTransformer(is_null, validate=False)

text = 'message'
text_transform = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                           ('best', TruncatedSVD(n_components=50))])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
                                ('cat', categorical_transform, categorical_features),
                                ('orig', original_transform, original),
                                ('tfidf', text_transform, 'message')
                                              ])

In [ ]:
preprocessor.fit_transform(X_2)

In [ ]:
clf = forest_clf

In [ ]:
pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('best', TruncatedSVD(n_components=100)),
                    ('clf', MultiOutputClassifier(clf))
                      ])

In [ ]:
# test to make sure this works ok
pipeline_2.fit(X_train, Y_train)

In [ ]:
print(pipeline_2.steps)

In [ ]:
evaluate_model(pipeline_2, X_test, Y_test)

In [ ]:
param_grid = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'best__n_components': (10, 50, 100),
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

In [ ]:
cv_2 = GridSearchCV(pipeline_2, param_grid, cv=3, verbose=10, n_jobs=-1)

In [ ]:
cv_2.fit(X_train, Y_train)

In [ ]:
from pprint import pprint

In [ ]:
pprint(cv_2.best_estimator_.steps)

In [ ]:
evaluate_model(cv_2.best_estimator_, X_test, Y_test)

### 9. Export your model as a pickle file

In [ ]:
import joblib

In [ ]:
joblib.dump(cv_2.best_estimator_, 'disaster_model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.